# Monte Carlo ETA Risk Simulation

This notebook estimates ETA uncertainty for a planned voyage using Monte Carlo simulation.

We compute:
- Mean ETA
- Percentiles: P50, P90, P95
- Probability of late arrival beyond a threshold


In [ ]:
from src.simulation import simulate_eta_hours
from src.risk_metrics import summarize_risk
from src.plotting import plot_eta_distribution


## Define voyage scenario

Change the distance/speed/threshold to explore different risk profiles.


In [ ]:
distance_nm = 1200
expected_speed_knots = 14
late_threshold_hours = 110
n_sims = 20000


In [ ]:
samples = simulate_eta_hours(
    distance_nm=distance_nm,
    expected_speed_knots=expected_speed_knots,
    n_sims=n_sims,
    seed=7
)

len(samples), samples[:5]


In [ ]:
summary = summarize_risk(samples, late_threshold_hours=late_threshold_hours)
summary


In [ ]:
plot_eta_distribution(samples, save_path="reports/figures/eta_hist.png")
print("Saved figure: reports/figures/eta_hist.png")


## Interpretation

- **Mean**: expected ETA under uncertainty
- **P95**: conservative ETA for planning (only ~5% of runs exceed this)
- **Probability of late arrival**: risk of breaching an SLA threshold

These metrics convert uncertainty into operational decision-making.


# Stress Scenarios

We compare a normal day vs adverse conditions using simple stress assumptions.


In [ ]:
normal = simulate_eta_hours(distance_nm, expected_speed_knots, n_sims=20000, seed=1)

# Storm: lower expected speed proxy (represents adverse sea/weather)
storm = simulate_eta_hours(distance_nm, expected_speed_knots * 0.90, n_sims=20000, seed=2)

# Port strike/congestion: even lower effective speed proxy
port_strike = simulate_eta_hours(distance_nm, expected_speed_knots * 0.85, n_sims=20000, seed=3)

summaries = {
    "normal": summarize_risk(normal, late_threshold_hours),
    "storm": summarize_risk(storm, late_threshold_hours),
    "port_strike": summarize_risk(port_strike, late_threshold_hours),
}
summaries


In [ ]:
import pandas as pd
pd.DataFrame(summaries).T


### Takeaway

Compare **P95 ETA** and **Probability of late arrival** across scenarios to quantify operational risk under stress.
